In [2]:
import pandas as pd
import os

In [10]:
working_dir = os.getcwd()

In [11]:
ext = r'\adult.data.csv'

In [72]:
data = pd.read_csv(working_dir+ext)

In [76]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [77]:
# Read data from file
df = data

In [78]:
# How many of each race are represented in this dataset? This should be a Pandas series with race names as the index labels.
race_count = df['race'].nunique()
print(race_count)

5


In [79]:
# What is the average age of men?
average_age_men = df[df['sex']=='Male']['age'].mean()

In [80]:
average_age_men

39.43354749885268

In [81]:
# What is the percentage of people who have a Bachelor's degree?
percentage_bachelors = len(df[df['education']=='Bachelors'])/len(df)

In [82]:
percentage_bachelors

0.16446055096587942

What percentage of people with advanced education (`Bachelors`, `Masters`, or `Doctorate`) make more than 50K?

What percentage of people without advanced education make more than 50K?

In [83]:
# with and without `Bachelors`, `Masters`, or `Doctorate`
adv_degree = df['education'].isin(['Bachelors', 'Masters', 'Doctorate'])

higher_education = len(df[adv_degree])/len(df)
lower_education = len(df[~adv_degree])/len(df)

In [84]:
df['salary'].unique()

array(['<=50K', '>50K'], dtype=object)

In [111]:
# percentage with salary >50K
rich = df['salary'] == '>50K'

higher_education_rich = len(df[adv_degree & rich])/len(df)
lower_education_rich = len(df[~adv_degree & rich])/len(df)

In [99]:
# What is the minimum number of hours a person works per week (hours-per-week feature)?
min_work_hours = df['hours-per-week'].min()

In [117]:
# What percentage of the people who work the minimum number of hours per week have a salary of >50K?
min_workers = df[(df['hours-per-week'] == min_work_hours)]
num_min_workers = len(min_workers)

rich_percentage = num_min_workers/len(df)

In [119]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [145]:
# percentage of people that earn >50k, by country
rich_countries = df[rich].groupby(['native-country'])['age'].count().reset_index().rename(columns={'age':'rich_country_pop'})
rich_countries.head()

,native-country,rich_country_pop
0,?,146
1,Cambodia,7
2,Canada,39
3,China,20
4,Columbia,2


In [142]:
country_pop = df.groupby(['native-country']).count().reset_index()[['native-country','age']].rename(columns={'age':'country_pop'})
country_pop.head()

,native-country,country_pop
0,?,583
1,Cambodia,19
2,Canada,121
3,China,75
4,Columbia,59


In [149]:
country_df = pd.merge(rich_countries,country_pop,on=['native-country'])
country_df['country_rich_pct'] = country_df['rich_country_pop']/country_df['country_pop']
country_df.head()

,native-country,rich_country_pop,country_pop,country_rich_pct
0,?,146,583,0.250429
1,Cambodia,7,19,0.368421
2,Canada,39,121,0.322314
3,China,20,75,0.266667
4,Columbia,2,59,0.033898


In [165]:
country_df.loc[np.where(country_df['country_rich_pct']==country_df['country_rich_pct'].max())[0][0]]['native-country']

'Iran'

In [166]:
country_df.loc[np.where(country_df['country_rich_pct']==country_df['country_rich_pct'].max())[0][0]]['country_rich_pct']

0.4186046511627907

In [167]:
# What country has the highest percentage of people that earn >50K?
highest_earning_country = country_df.loc[np.where(country_df['country_rich_pct']==country_df['country_rich_pct'].max())[0][0]]['native-country']
highest_earning_country_percentage = country_df.loc[np.where(country_df['country_rich_pct']==country_df['country_rich_pct'].max())[0][0]]['country_rich_pct']

In [173]:
df[rich&(df['native-country']=='India')].groupby(['occupation'])['age'].count().reset_index().max()['occupation']

'Transport-moving'

In [ ]:
# Identify the most popular occupation for those who earn >50K in India.
top_IN_occupation = df[rich&(df['native-country']=='India')].groupby(['occupation'])['age'].count().reset_index().max()['occupation']